<a href="https://colab.research.google.com/github/paluigi/COICOP-comparison/blob/master/Data_Scraping_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Scrapy to get online groceries data

Reference from [this webpage](https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html) for generic scraping using Jupyter notebooks.
Reference from [this project](https://github.com/limi7break/esselunga-scrape/blob/master/esselunga.ipynb) for Esselunga-specific strategies.

List of Online shopping websites for scraping:


1.   [Esselunga](https://www.esselungaacasa.it)
2.   [Coop](https://www.easycoop.com/)
3.   [Iperal](https://www.iperalspesaonline.it/)
4.   [Ali](https://www.aliperme.it/)
5.   [Spesasprint](https://www.spesasprint.it/)
6.   [Tulips](https://www.tulipsmarket.com)
7.   [Todis](https://todisacasa.it/)
8.   [Cicalia](https://www.cicalia.com/it/)
9.   [Spesati](https://www.spesati.it)
10.  [Farina Market](https://farinamarket.gospesa.it/)




Initial test is: [Farina Market](https://farinamarket.gospesa.it/) as it seems easier to scrap data. 

Other shops on Gospesa platform at [this link](https://gospesa.it/i-punti-vendita-gospesa/)

Setup the notebook

In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.9'

Import Scrapy and other libraries (install if library is not present)

In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

This class creates a simple pipeline that writes all found items to a JSON file, where each line contains one JSON element.

In [89]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

The PSpider class defines from which URLs to start crawling and which values to retrieve. I set the logging level of the crawler to warning, otherwise the notebook is overloaded with DEBUG messages about the retrieved data.

In [90]:
import logging
import datetime


class PSpider(scrapy.Spider):
    name = "prices"
    start_urls = ['https://farinamarket.gospesa.it/14917-pasta', 
                  'https://ipermercatomontenero.gospesa.it/14917-pasta', 
                  'https://ipermercatolaquilone.gospesa.it/14917-pasta', 
                  'https://carrefour.gospesa.it/pescara/14917-pasta']
    
    custom_settings = {
          'LOG_LEVEL': logging.WARNING,
          'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
         # 'FEED_FORMAT':'json',                                 # Used for pipeline 2
         # 'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
      }

    def parse(self, response):
      yield{
          'name': response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "tvproduct-title", " " ))]//a//text()').extract(),
          'price': response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "tvnew-price", " " ))]//text()').re('[,0-9]+'), # Regular expression removes Euro sign and spaces
          'url': response.url,
          'day': str(datetime.datetime.now()),
      }
        


Define a function to start the spider multiple times

In [91]:
from multiprocessing import Process, Queue
from twisted.internet import reactor
import scrapy.crawler as crawler

def run_spider(spider):
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

Start the spider

In [92]:
run_spider(PSpider)

In [93]:
ll quoteresult.*


-rw-r--r-- 1 root 4674 Jul 24 11:33 quoteresult.jl


In [94]:
!tail -n 3 quoteresult.jl

{"name": ["alce nero Conchiglie Grano Duro 500 g", "alce nero Farfalle Grano Duro 500 g", "alce nero Farro Penne Rigate 500 g", "alce nero Farro Tagliatelle 250 g", "alce nero Fusilli Farro 500 g", "alce nero Fusilli Grano Duro 500 g", "alce nero Fusilli Integrali Farro 500 g", "alce nero Penne Rigate Grano Duro 500 g", "alce nero Penne Rigate Integrali Farro 500 g", "alce nero Penne Rigate Integrali Khorasan 500 g", "alce nero Penne Rigate Khorasan 500 g", "alce nero Semolino Senatore Cappelli Biologico 250 g", "alce nero Spaghetti Farro 500 g", "alce nero Spaghetti Grano Duro 500 g", "alce nero Spaghetti Integrali Farro 500 g", "alce nero Spaghetti Integrali Khorasan 500 g", "alce nero Spaghetti Khorasan 500 g", "alce nero Stelline di Farro Biologiche 500 g", "alce nero Tagliatelle Grano Duro 250 g", "alce nero Tubetti Farro 500 g", "alce nero Variet\u00e0 di Grano Senatore Cappelli Fusilloni 500 g", "alce nero Variet\u00e0 di Grano Senatore Cappelli Penne Rigate 500 g", "alce nero V

In [73]:
import pandas as pd
dfjl = pd.read_json('quoteresult.jl', lines=True)
dfjl

,name,price,url,day
0,"[alce nero Farro Penne Rigate 500 g, alce nero...","[3,66, 2,26, 3,66, 4,06, 3,34, 5,08, 5,08, 4,7...",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
1,"[alce nero Conchiglie Grano Duro 500 g, alce n...","[1,74, 1,74, 3,49, 2,13, 3,49, 1,74, 2,95, 1,7...",https://ipermercatomontenero.gospesa.it/14917-...,2020-07-24 11:25:19.359440
2,"[Penne Rigate Carrefour 500gr, Spaghetti Carre...","[0,55, 0,55, 0,55, 0,55, 0,55, 0,55, 0,55, 0,5...",https://carrefour.gospesa.it/pescara/14917-pasta,2020-07-24 11:25:19.435238
3,"[alce nero Farro Penne Rigate 500 g, alce nero...","[3,49, 2,13, 3,49, 2,79, 2,79, 4,79, 4,58, 3,4...",https://farinamarket.gospesa.it/14917-pasta,2020-07-24 11:25:20.000929


Format the data into a dataframe, one line per each product

In [109]:
price_list = []
for shop in dfjl.index:
  for i, product in enumerate(dfjl.at[shop, "name"]):
    price_list.append([product, dfjl.at[shop, "price"][i], dfjl.at[shop, "url"], dfjl.at[shop, "day"] ])

prices = pd.DataFrame(price_list, columns =["product","price", "url", "date" ])
prices

,product,price,url,date
0,alce nero Farro Penne Rigate 500 g,"3,66",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
1,alce nero Farro Tagliatelle 250 g,"2,26",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
2,alce nero Fusilli Farro 500 g,"3,66",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
3,alce nero Penne Rigate Integrali di Grano Duro...,"4,06",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
4,alce nero Penne Rigate Integrali Farro 500 g,"3,34",https://ipermercatolaquilone.gospesa.it/14917-...,2020-07-24 11:25:19.221987
...,...,...,...,...
75,alce nero Tubetti Farro 500 g,"3,49",https://farinamarket.gospesa.it/14917-pasta,2020-07-24 11:25:20.000929
76,alce nero Varietà di Grano Senatore Cappelli F...,"2,39",https://farinamarket.gospesa.it/14917-pasta,2020-07-24 11:25:20.000929
77,alce nero Varietà di Grano Senatore Cappelli P...,"2,39",https://farinamarket.gospesa.it/14917-pasta,2020-07-24 11:25:20.000929
78,alce nero Varietà di Grano Senatore Cappelli R...,"2,39",https://farinamarket.gospesa.it/14917-pasta,2020-07-24 11:25:20.000929
